In [51]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split

In [52]:
data = pd.read_csv('Real_Combine.csv') # Air Quality index prediction

In [53]:
data.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [54]:
# Regression Problem

X = data.iloc[:,0:8]
y = data.iloc[:,-1]

In [55]:
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers',2,20)):
        model.add(Dense(units = hp.Int('units_'+str(i), min_value = 32, max_value = 512, step = 32), activation = 'relu'))
        model.add(Dense(1, activation = 'linear')) # For regression, output will be 1 and activation function will be linear
        model.compile(
            optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), 
            loss = 'mean_absolute_error',
            metrics = ['mean_absolute_error'])
    return model

In [56]:
tuner = RandomSearch(
        build_model,
        objective = 'val_mean_absolute_error',
        max_trials = 5,
        executions_per_trial = 3,
        overwrite=True,
        directory = 'project',
        project_name = 'Air Quality Index')

In [57]:
tuner.search_space_summary() 
#the number of filters for the convolutional layers. the number of units for the dense layer. its activation function.

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}


In [58]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)

In [59]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 5 Complete [00h 00m 18s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 01m 05s
INFO:tensorflow:Oracle triggered exit
